In [ ]:
!pip install openai
!pip install transformers
!pip install datasets

In [ ]:
import sys
sys.path.append('./../') # evaluatorディレクトリのあるパスを指定

In [ ]:
OUTPUT_FILENAME = "stablelm_7b"

In [ ]:
import pandas as pd
# Load questions with responses
# responses = pd.read_csv('./../datasets/Example/sample_questions_GPT4_response.csv')
responses = pd.read_csv('stablelm_alpha_7b_instruct_response.csv')

In [ ]:
# responses = responses[['id', 'question', 'GPT4_response']]

In [ ]:
responses.head(5)

In [ ]:
from do_not_answer.evaluator import construct_message, gpt, parse_labels

In [ ]:
# 角括弧、波括弧がresponseに含まれていると後続のconstruct_message()でエラーが発生することがあるため除去

col_name = "stablelm_alpha_7b_instruct_response" # responseが格納されている列名

responses[col_name] = responses[col_name].fillna("")

for n in range(len(responses)):
    responses[col_name][n] = responses[col_name][n].replace("[", "")
    responses[col_name][n] = responses[col_name][n].replace("]", "")
    responses[col_name][n] = responses[col_name][n].replace("{", "")
    responses[col_name][n] = responses[col_name][n].replace("}", "")

In [ ]:
messages = construct_message(responses)

In [ ]:
messages[0]

In [ ]:
import openai
import time

openai.api_key = "xxxxxxxxxxxxxx" # enter your openai key

responses_do_not_answer = {}

for i, message in enumerate(messages):
    time.sleep(1)

    if i%25==0 and i>0: # 25リクエスト毎にバックアップのためリストを書き出し
        print(i)
        pd.DataFrame.from_dict(responses_do_not_answer, columns=["response"], orient='index').to_csv('temp_response_annotated.csv', index=False)
        time.sleep(10)
        
    r = gpt(message, model="gpt-4-0613")
    responses_do_not_answer[i] = r

In [ ]:
responses_do_not_answer

In [ ]:
# reviews = [responses_do_not_answer[i] for i in range(5)]
reviews = [responses_do_not_answer[i] for i in range(len(responses))]

In [ ]:
labels = parse_labels(reviews)
labels

In [ ]:
responses["reviews"] = reviews
responses["action"] = labels
responses

In [ ]:
responses["action"].mean()

In [ ]:
# responses.to_csv("./../datasets/Example/sample_questions_GPT4_response_annotated.csv", index=None)
responses.to_csv(f"{OUTPUT_FILENAME}_response_annotated.csv", index=None)